## Case Study:
## Building ETL Pipelines to Integrate Property Data of Zillow and Realtor

### Goals

Write code (python preferred) to integrate the two datasets, considering data quality, consistency, and scalable ETL process design.
Convert the attached JSON files into relational tables in an Excel file. Key tables should include property locations, descriptions, tax history, etc.

Identify data issues and give solutions to fix them.

Integrate the two datasets and remove duplicate information. Note that there can be similar columns with different names, for example, Bath and Bathroom.

Compare your output data with the attached schema template and identify missing attributes.

### Architecture

Since startups always place a focus on the iterative ability on the architecture, ***snowflake*** schema is perferred to achieve both iterative ability and logical isolation (expect to expand in future following the Kimball Methodology)

For this case study, I chose to construct a fact table for tax history assessment ***factTaxHistory***, a main dimensional table for properties ***dimProperties***, and subdimensional tables for addresses, descriptions and utilities ***dimAddress***, ***dimDescription***, ***dimUtility***.

I utilized ***Spark*** to implement the ETL processes to extract and integrate the source data of scraped json files from ***Zillow*** and ***Realtor***, transform the data into a relational table schema, load into the destination of an excel file. (In practical cases, insert or upsert actions should be operated to merge data into a data warehouse)

### Data Consistency

To keep data consistency among different data sources, there are approaches to set feature vectors for each record and common ML methods (such as GlueETL FindMatches) to deduplicate. However, in this case, the pair of latitude and longitude could be used to identify a unique property. We can prove that through merging by or deduplicating by the (lat, lon) pair of rounded-up values to 3 decimals, we can set a control that property records with distance detected within 80 metres are recognized as the same property. (Or more strictly, choosing to round up to 4 decimals leads to a limit of 8 metres)

**Haversine Formula**

We can use latitude and longitude to identify the properties by using (lat, lon) pair.

We can prove that when we have a maximum bias of $10^{-4}$ for both lat and lon, we will control the distance of two places under 10 meters; and when we have a maximum bias of $10^{-3}$ for both lat and lon, we will control the distance of two places under 80 meters. So, by rounding up the latitude and longitude to four decimal places, we can deduplicate by (lat, lon) pair to ensure two properties are at least in the same building. So we can deduplicate by (lat, lon) pair plus some other attributes of properties.

The Haversine formula is used to calculate the great-circle distance between two points on a sphere given their longitudes and latitudes. It is given by:

$$
d = 2r \arcsin\left(\sqrt{\sin^2\left(\frac{\Delta\phi}{2}\right) + \cos(\phi_1)\cos(\phi_2)\sin^2\left(\frac{\Delta\lambda}{2}\right)}\right)
$$

where:
- $d$ is the distance between the two points,
- $r$ is the radius of the sphere (e.g., the Earth's radius),
- $\phi_1$ and $\phi_2$ are the latitudes of the two points in radians,
- $\Delta\phi$ is the difference in latitudes in radians,
- $\Delta\lambda$ is the difference in longitudes in radians.

when $\Delta\phi\rightarrow0$ and $\Delta\lambda\rightarrow0$ we have $\sin^2\left(\frac{\Delta\phi}{2}\right) + \cos(\phi_1)\cos(\phi_2)\sin^2\left(\frac{\Delta\lambda}{2}\right)\rightarrow0$ and $\sin^2\left(\frac{\Delta\phi}{2}\right) + \cos(\phi_1)\cos(\phi_2)\sin^2\left(\frac{\Delta\lambda}{2}\right)=\frac{\Delta\phi^{2}}{4}+\frac{\Delta\lambda^{2}}{4}+O(\Delta\phi^{4})+O(\Delta\lambda^{4})$

Since we control $\Delta\phi$ and $\Delta\lambda$ in the same precision, we can write them as the same as $\Delta\phi$, the previous one becomes: $\frac{\Delta\phi^{2}}{2}+O(\Delta\phi^{4})$
For function of $\arcsin(\sqrt{x^{2}})=\arcsin x = x + o(x^2)$

By setting $x=\Delta\phi(1+\Delta\phi)$ we have $\arcsin(\sqrt{\Delta\phi^{2}+2\Delta\phi^{3}+\Delta\phi^{4}})=\Delta\phi+\Delta\phi^2+o(\Delta\phi^{2})$

Then $\arcsin(\sqrt{\frac{\Delta\phi^{2}}{2}+O(\Delta\phi^{4})})\le\arcsin(\sqrt{\frac{\Delta\phi^{2}+2\Delta\phi^{3}+\Delta\phi^{4}}{2}})=\frac{1}{\sqrt{2}}\Delta\phi+\frac{1}{\sqrt{2}}\Delta\phi^2+o(\Delta\phi^{2})$

We have $\Delta\phi\leq 10^{-4}*\frac{2\pi}{360}$ then $d=2*r*(\frac{1}{\sqrt{2}}\Delta\phi+\frac{1}{\sqrt{2}}\Delta\phi^2+o(\Delta\phi^{2}))\approx d=2*r*(\frac{1}{\sqrt{2}}\Delta\phi)\leq \frac{6378.1\times10^{3}}{\sqrt{2}}\times(10^{-4}*\frac{2\pi}{360})\ meters\approx7.87\ meters$

Considering there an bias factor for Haversine Formula up to $0.5\%$, we have $d\leq 7.87*(1+0.5\%)=7.91\ meters$

Besides, when we control $\Delta\phi\leq 10^{-3}*\frac{2\pi}{360}$, we have $d\leq 79.1\ meters$

### Some strategies to handle the Data Quality Issues

- Address features such as county and FIPS code are not ensured to exist.
    - Use files from [www.huduser.gov](https://www.huduser.gov/portal/datasets/usps_crosswalk.html) to infer the missing values from existing address features.
- In the Realtor dataset, property features such as area, price, bedrooms, bathrooms are not ensured to exist when the struct feature of units exist.
    - Use sum of living area, listing price, bedroom number, bathroom number for the units as the feature of the property.
- When identifying the same property, some features are recorded differently.
    - Use features extracted from Zillow prioritized over features from Realtor.
- Tax histories are not ensured to be recorded for each property.
    - By building a fact table for tax histories, we focus on existing tax histories.
- Boolean values such as flags indicating the existence of a garage, a pool, a cooling system or a heating system are very sparse.
    - Null values are transformed into False values.
- Area features such as living area and lot size could be in different units.
    - Store the features in string types with the unit concatenated.
- For categorical types, there are values representing the same type but with minor difference in different data sources.
    - While edit distance could be useful in this case, a formal way is to create a standardized dictionary and transform original values.

### Results

In [21]:
import pandas as pd

file_path = 'result_table.xlsx'
df_TaxHistory = pd.read_excel(file_path, sheet_name='factTaxHistory', header=0, index_col=0)
df_Property = pd.read_excel(file_path, sheet_name='dimProperty', header=0, index_col=0)
df_Address = pd.read_excel(file_path, sheet_name='dimAddress', header=0, index_col=0)
df_Description = pd.read_excel(file_path, sheet_name='dimDescription', header=0, index_col=0)
df_Utility = pd.read_excel(file_path, sheet_name='dimUtility', header=0, index_col=0)

In [22]:
df_TaxHistory

,propertyId,taxYear,taxPaid,value,taxHistoryId
0,1,2023,11392,456311,1
1,1,2022,11671,377301,2
2,1,2021,8401,316422,3
3,1,2020,6786,316422,4
4,1,2019,6950,330851,5
...,...,...,...,...,...
137,16,2020,1018357,53139653,138
138,16,2019,1330398,52941572,139
139,16,2018,1296826,50494300,140
140,16,2017,1334844,45284840,141


In [23]:
df_Property

,propertyID,addressId,descriptionID,utilityID
0,1,4,5,1
1,2,13,13,5
2,3,2,10,9
3,4,12,14,8
4,5,1,16,7
5,6,8,3,10
6,7,9,6,3
7,8,11,7,13
8,9,15,9,2
9,10,14,11,11


In [24]:
df_Address

,state,countyFIPS,city,streetAddress,zipcode,latitude,longitude,addressId
0,TX,48453,Austin,8715 W Highway 71 #4_4206,78735,30.250511,-97.896490,1
1,TX,48453,Austin,6508 Steep Cactus Trl #20,78735,30.244486,-97.877050,2
2,TX,48453,Austin,2601 Scofield Ridge Pkwy,78727,30.428510,-97.697044,3
3,TX,48453,Austin,7603 Woodstone Cv,78749,30.206296,-97.835752,4
4,TX,48491,Austin,10800 Lakeline Blvd,78717,30.482453,-97.793530,5
5,TX,48453,Austin,2601 Scofield Ridge Pkwy,78727,30.430732,-97.697007,6
6,TX,48453,Austin,9906 Hundred Oaks Cir Unit A,78750,30.421557,-97.800574,7
7,TX,48453,Austin,403 E 32nd St Unit B,78705,30.294802,-97.733620,8
8,TX,48453,Austin,6505 Cat Creek Trl,78731,30.360643,-97.779975,9
9,TX,48491,Austin,8100 Anderson Mill Rd,78729,30.461880,-97.757279,10


In [26]:
df_Description

,homeType,homeStatus,textDescription,yearBuilt,price,livingArea,lotSize,descriptionId
0,duplex_triplex,for_rent,"Rare find of a 3 Bed 2 bath, with private yard...",1983.0,1725.0,1914 sqft,25700 sqft,1
1,apartment,other,Great location: close to domain and mopac expr...,1999.0,117594538.0,711 sqft,26.37 Acres,2
2,single_family,for_rent,Available NOW! LEASE THROUGH JULY 2024 ONLY! A...,1934.0,1600.0,630 sqft,7492 sqft,3
3,apartment,for_rent,Better-than-new apartments situated north of d...,2015.0,7925.0,4344 sqft,16.96 Acres,4
4,multi_family,sold,"This is a 2070 square foot, 2.0 bathroom, apar...",1983.0,542231.0,2070 sqft,9091 sqft,5
5,duplex_triplex,for_rent,"West side duplex in a hilly, wooded neighborho...",1983.0,1995.0,1178 sqft,17454 sqft,6
6,apartment,for_rent,Welcome to The Lucent Apartments located in Au...,2021.0,10933.0,6543 sqft,17.58 Acres,7
7,duplex_triplex,for_rent,"This is a 1632 square foot, 2.0 bathroom, apar...",1984.0,1500.0,816 sqft,6630 sqft,8
8,apartment,other,"This is a 287272 square foot, apartment home. ...",2021.0,88483962.0,287272 sqft,17.68 Acres,9
9,other,other,This 1481 square foot townhome home has 3 bedr...,NaN,417900.0,1481 sqft,NaN,10


In [27]:
df_Utility

,bathrooms,bedrooms,hasGarage,hasCooling,hasHeating,hasPrivatePool,utilityId
0,2,0,False,False,False,False,1
1,0,0,False,False,False,False,2
2,2,2,False,False,False,False,3
3,2,3,False,False,False,False,4
4,0,9,False,False,False,False,5
5,11,11,False,False,False,False,6
6,2,2,False,True,True,False,7
7,1,1,False,False,False,False,8
8,0,3,False,True,False,False,9
9,1,2,False,False,False,False,10


### Insights into the ATTOM Schema

As a standard schema for the properties, the ATTOM Schema has evolved for tens of years and has been elaborately designed for storing detailed information in different groups of topics. Some points can be learned from the perspective of a startup to build a data warehouse:
- Abundant domain knowledge of properties could be learned from the ATTOM schema. From the perspective of business intelligence, valuable property information includes legal information, party owners, contact owners, deed owners, etc.
- While the ATTOM schema keeps heavy data redundancy for the convenience of business analyses (such as splitting street address into several columns, keeping isolated sections for information of attic, parking, garage), it somehow tends to guide the future trends of property data schema. However, from the view of a startup company, heavy data redundancy could lead to low data quality (such as data sparsity) and a burden of infrastructure cost (compute and storage).
- The ATTOM schema sets strict data types for each column. But startups focus more on the iterative ability of the architecture. In the early phase of development, it should be avoided that excessive effort is put into optimization.